<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/neox1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [ ]:
!apt-get update -y
!apt-get install python3.8 python3.8-dev
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!python --version
!apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3.8 get-pip.py
!python3.8 -m pip --version
!update-alternatives --install /usr/bin/pip3 pip3 /usr/bin/python3.8 1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [6]:
!git clone https://github.com/EleutherAI/gpt-neox.git
%cd /content/gpt-neox

fatal: destination path 'gpt-neox' already exists and is not an empty directory.
/content/gpt-neox


In [7]:
!ls

CITATION.cff		      docker-compose.yml  images	 prepare_data.py  tools
configs			      Dockerfile	  LICENSE	 README.md	  train.py
CONTRIBUTING.md		      eval.py		  MANIFEST.in	 README-MUP.md
deepy.py		      eval_tasks	  megatron	 requirements
docker-compose-dockerhub.yml  generate.py	  post-training  tests


In [4]:
!pip install tokenizers==0.18.0
!pip install transformers==4.38.0 tokenizers<0.19
!pip install datasets==3.0.2 fsspec==2024.9.0 tqdm==4.66.3 requests==2.32.2
!pip install evaluate==0.4.3 tqdm==4.66.3
!pip install tiktoken==0.7.0 requests>=2.26.0

ERROR: Could not find a version that satisfies the requirement tokenizers==0.18.0 (from versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.7.0rc1, 0.7.0rc2, 0.7.0rc3, 0.7.0rc4, 0.7.0rc5, 0.7.0rc6, 0.7.0rc7, 0.7.0, 0.8.0.dev0, 0.8.0.dev1, 0.8.0.dev2, 0.8.0rc1, 0.8.0rc2, 0.8.0rc3, 0.8.0rc4, 0.8.0, 0.8.1rc1, 0.8.1rc2, 0.8.1, 0.9.0.dev0, 0.9.0.dev1, 0.9.0.dev2, 0.9.0.dev3, 0.9.0.dev4, 0.9.0rc1, 0.9.0rc2, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.10.0rc1, 0.10.0, 0.10.1rc1, 0.10.1, 0.10.2, 0.10.3, 0.11.0, 0.11.1, 0.11.2, 0.11.3, 0.11.4, 0.11.5, 0.11.6, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3rc1, 0.13.3, 0.13.4rc1, 0.13.4rc2, 0.13.4rc3, 0.14.0rc1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2rc1, 0.15.2, 0.19.0, 0.19.1, 0.20.0, 0.20.1rc1, 0.20.1)
ERROR: No matching distribution found for tokenizers==0.18.0
/bin/bash: line 1: 0.19: No such file or directory


In [7]:
!pip install -r requirements/requirements.txt

In [8]:
import requests
from pathlib import Path

# Download Shakespeare data
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
Path("shakespeare.txt").write_text(response.text)

# Split text into chunks
with open("shakespeare.txt") as f:
    text = f.read()

chunk_size = 256
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Save as JSONL format
with open("shakespeare.jsonl", "w") as f:
    for chunk in chunks:
        f.write(f'{{"text": "{chunk}"}}\n')

In [9]:
from tokenizers import ByteLevelBPETokenizer

import os

# Create the directory if it doesn't exist
os.makedirs("tokenizer", exist_ok=True)

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["shakespeare.txt"], vocab_size=5000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [10]:
import yaml

# Define the configuration data as a dictionary
config_data = {
    "vocab-file": "shakespeare_tokenizer.json",
    "hidden-size": 384,
    "num-attention-heads": 6,
    "num-layers": 6,
    "max-position-embeddings": 256,
    "micro-batch-size": 32,
    "seq-length": 256,
    "train-iters": 5000,
    "lr-decay-iters": 5000,
    "lr": 6e-4,
    "min-lr": 6e-5,
    "warmup": 0.01,
    "adam-beta1": 0.9,
    "adam-beta2": 0.95,
    "weight-decay": 0.1,
    "clip-grad": 1.0,
    "fp16": True
}

# Write the dictionary to a YAML file
with open("shakespeare_config.yml", "w") as f:
    yaml.dump(config_data, f, default_flow_style=False)


In [11]:
%run deepy.py train.py -d configs shakespeare_config.yml

ModuleNotFoundError: No module named 'deepspeed'